# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [140]:
import warnings
warnings.filterwarnings('ignore')

In [141]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import google_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [142]:
# The path to our CSV file
csv_path = "cities.csv"
city_df = pd.read_csv(csv_path)

city_df.head()

,city,cloudiness,country,date,humidity,lat,Lng,Temp,Wind
0,Strezhevoy,90,RU,1579385670,92,60.73,77.59,8.60,4.47
1,Kapaa,75,US,1579385671,60,22.08,-159.32,75.20,17.22
2,Yangambi,89,CD,1579385672,92,0.81,24.43,71.02,2.71
3,Saint-Philippe,75,RE,1579385673,83,-21.36,55.77,80.60,5.82
4,Achacachi,100,BO,1579385674,94,-16.05,-68.68,38.28,9.64


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [143]:
# Store latitude and longitude in locations
locations = city_df[["lat", "Lng"]]

# Fill NaN values and convert to float
humidity = city_df["humidity"]

In [144]:
 # Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=city_df.humidity.max(),
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [172]:
ideal_city_df = city_df.query('Temp < 80 & Temp >= 70 & Wind < 10 & cloudiness <= 10 & humidity <= 55')
ideal_city_df

,city,cloudiness,country,date,humidity,lat,Lng,Temp,Wind
74,Kaeo,0,NZ,1579385752,52,-35.10,173.78,75.99,5.99
137,Chui,7,UY,1579385665,49,-33.70,-53.46,77.27,5.91
253,Niamey,0,NE,1579385948,19,13.51,2.11,71.60,6.93
304,Leer,1,SS,1579386004,18,8.30,30.14,74.84,6.58
378,Biankouma,0,CI,1579386085,29,7.74,-7.61,74.59,2.30
384,Kita,0,ML,1579386092,15,13.03,-9.49,75.18,5.91
423,Ouadda,0,CF,1579386135,23,8.08,22.40,70.88,6.53
426,Ouahigouya,0,BF,1579386138,17,13.58,-2.42,70.12,9.37
523,Kibala,8,TD,1579386245,40,9.11,18.35,73.40,6.93
524,Bria,0,CF,1579386246,25,6.54,21.99,74.05,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [173]:
hotel_df = ideal_city_df
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Address"] = ""
hotel_df["Hotel Rating"] = ""
hotel_df

,city,cloudiness,country,date,humidity,lat,Lng,Temp,Wind,Hotel Name,Hotel Address,Hotel Rating
74,Kaeo,0,NZ,1579385752,52,-35.10,173.78,75.99,5.99,,,
137,Chui,7,UY,1579385665,49,-33.70,-53.46,77.27,5.91,,,
253,Niamey,0,NE,1579385948,19,13.51,2.11,71.60,6.93,,,
304,Leer,1,SS,1579386004,18,8.30,30.14,74.84,6.58,,,
378,Biankouma,0,CI,1579386085,29,7.74,-7.61,74.59,2.30,,,
384,Kita,0,ML,1579386092,15,13.03,-9.49,75.18,5.91,,,
423,Ouadda,0,CF,1579386135,23,8.08,22.40,70.88,6.53,,,
426,Ouahigouya,0,BF,1579386138,17,13.58,-2.42,70.12,9.37,,,
523,Kibala,8,TD,1579386245,40,9.11,18.35,73.40,6.93,,,
524,Bria,0,CF,1579386246,25,6.54,21.99,74.05,4.70,,,


In [177]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": google_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
   
    # convert to json
    name_address = name_address.json()
  
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        hotel_df.loc[index, "Hotel Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print(f"{hotel_df.city[index]} Missing field/result... skipping.")
        hotel_df = hotel_df.drop([index], axis=0)

In [178]:
hotel_df

,city,cloudiness,country,date,humidity,lat,Lng,Temp,Wind,Hotel Name,Hotel Address,Hotel Rating
74,Kaeo,0,NZ,1579385752,52,-35.10,173.78,75.99,5.99,Pacific Harbour Lodge,"Harbour, Northland 8 Lewer Rd, Whangaroa",4.6
137,Chui,7,UY,1579385665,49,-33.70,-53.46,77.27,5.91,Hotel Internacional,"Av. Brasil 679, Chuy",4.1
253,Niamey,0,NE,1579385948,19,13.51,2.11,71.60,6.93,"Radisson Blu Hotel & Conference Center, Niamey","Quartier Plateau, Bd. de la Republique, Niamey",4.6
378,Biankouma,0,CI,1579386085,29,7.74,-7.61,74.59,2.30,Hôtel Licolais,Biankouma,5
384,Kita,0,ML,1579386092,15,13.03,-9.49,75.18,5.91,HÔTEL LE RELAIS,"Kita route hôpital BP, 44",3.3
426,Ouahigouya,0,BF,1579386138,17,13.58,-2.42,70.12,9.37,Hotel de l'Amitie,Ouahigouya,3.6
523,Kibala,8,TD,1579386245,40,9.11,18.35,73.40,6.93,Hôtel des Chasses,Sarh,0
552,Koutiala,0,ML,1579386276,13,12.39,-5.46,72.10,9.26,Hotel Flasso 1,"San District Koko Extension, Koutiala",3.9


In [179]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "Lng"]]
nl = '\n'

In [180]:

# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [181]:
 # Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))